# TensorFlow Datasetのテスト

[tf.data.Dataset のAPIドキュメント (tensorflow.org/api_docs)](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)

In [1]:
import numpy as np
import tensorflow as tf

## 共通的に利用する関数定義

0..9までの連番を格納したDatasetを作成する`make_ds`と、Datasetの中身を表示する`print_ds`を定義。  
1回の`print_ds`呼び出しが、機械学習の1エポックのデータ取り出しに相当する。  

In [2]:
def make_ds():
    return tf.data.Dataset.range(10)


def print_ds(*args):
    line = ""
    for ds in args:
        data = list(ds.as_numpy_iterator())
        line += str(data)
    print(line)

In [3]:
ds = make_ds()
print_ds(ds)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


## Datasetに対する操作と結果

In [4]:
LOOPS = 12  # Datasetから値を取り出す回数（エポック数のイメージ）

### shuffle操作

In [5]:
# 単純なDatasetの場合、何度取り出しても同じ値になる。
ds = make_ds()
for i in range(LOOPS):
    print_ds(ds)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [6]:
# shuffleすると、取り出す度にデータがシャッフルされる。
# shuffleの引数にはデータ数を指定すると、データ全体がほぼ均一にシャッフルされる。
ds = make_ds().shuffle(10)
for i in range(LOOPS):
    print_ds(ds)

[2, 7, 3, 1, 6, 5, 4, 8, 0, 9]
[7, 2, 9, 4, 6, 3, 5, 8, 1, 0]
[0, 2, 3, 7, 1, 9, 8, 4, 5, 6]
[8, 1, 2, 9, 7, 6, 4, 3, 0, 5]
[4, 2, 9, 6, 5, 1, 0, 8, 3, 7]
[7, 5, 8, 4, 6, 0, 3, 9, 2, 1]
[5, 3, 2, 4, 8, 1, 6, 7, 9, 0]
[6, 9, 4, 1, 8, 0, 2, 5, 7, 3]
[3, 1, 9, 0, 6, 5, 7, 8, 2, 4]
[7, 4, 8, 5, 6, 0, 9, 1, 3, 2]
[8, 0, 9, 7, 3, 4, 5, 6, 1, 2]
[6, 0, 8, 5, 1, 7, 9, 4, 2, 3]


In [7]:
# shuffleに、reshuffle_each_iteration=Falseを設定すると、2回目以降は同じデータの並び順になる。
# つまり、最初の1回だけシャッフルされるような状態。
# デフォルトはreshuffle_each_iteration=Trueとなっているため、取り出す度にシャッフルされる。
ds = make_ds().shuffle(10, reshuffle_each_iteration=False)
for i in range(LOOPS):
    print_ds(ds)

[2, 3, 5, 9, 7, 4, 0, 6, 1, 8]
[2, 3, 5, 9, 7, 4, 0, 6, 1, 8]
[2, 3, 5, 9, 7, 4, 0, 6, 1, 8]
[2, 3, 5, 9, 7, 4, 0, 6, 1, 8]
[2, 3, 5, 9, 7, 4, 0, 6, 1, 8]
[2, 3, 5, 9, 7, 4, 0, 6, 1, 8]
[2, 3, 5, 9, 7, 4, 0, 6, 1, 8]
[2, 3, 5, 9, 7, 4, 0, 6, 1, 8]
[2, 3, 5, 9, 7, 4, 0, 6, 1, 8]
[2, 3, 5, 9, 7, 4, 0, 6, 1, 8]
[2, 3, 5, 9, 7, 4, 0, 6, 1, 8]
[2, 3, 5, 9, 7, 4, 0, 6, 1, 8]


### shuffleとcache

In [8]:
# cacheを指定するとメモリ上にデータをキャッシュできるため、2回目以降のデータ取得でパフォーマンス向上が期待できる。
# しかし、shuffle -> cache の順序で適用すると、2回目以降のシャッフルが反映されない。
# なお、2回目以降のシャッフルを抑止する目的では使用しない方が良い。
# 2回目以降のシャッフルを抑止するのであれば、shuffleでreshuffle_each_iteration=Falseを指定する。
ds = make_ds().shuffle(10).cache()  # この順番は非推奨
for i in range(LOOPS):
    print_ds(ds)

[8, 2, 9, 5, 4, 1, 7, 6, 3, 0]
[8, 2, 9, 5, 4, 1, 7, 6, 3, 0]
[8, 2, 9, 5, 4, 1, 7, 6, 3, 0]
[8, 2, 9, 5, 4, 1, 7, 6, 3, 0]
[8, 2, 9, 5, 4, 1, 7, 6, 3, 0]
[8, 2, 9, 5, 4, 1, 7, 6, 3, 0]
[8, 2, 9, 5, 4, 1, 7, 6, 3, 0]
[8, 2, 9, 5, 4, 1, 7, 6, 3, 0]
[8, 2, 9, 5, 4, 1, 7, 6, 3, 0]
[8, 2, 9, 5, 4, 1, 7, 6, 3, 0]
[8, 2, 9, 5, 4, 1, 7, 6, 3, 0]
[8, 2, 9, 5, 4, 1, 7, 6, 3, 0]


In [9]:
# cache -> shuffle という順序にすれば、正しくシャッフルが機能する。
# 2回目以降の取り出しでも、shuffleが正しく機能している。
ds = make_ds().cache().shuffle(10)  # 正しい順序
for i in range(LOOPS):
    print_ds(ds)

[1, 7, 9, 6, 4, 0, 2, 5, 3, 8]
[9, 6, 2, 3, 1, 5, 7, 4, 8, 0]
[6, 1, 2, 7, 9, 4, 5, 8, 0, 3]
[7, 2, 8, 9, 6, 5, 1, 4, 3, 0]
[5, 4, 7, 8, 9, 3, 0, 6, 2, 1]
[7, 6, 0, 9, 8, 2, 4, 3, 1, 5]
[8, 6, 7, 1, 4, 2, 9, 5, 3, 0]
[5, 9, 2, 1, 3, 8, 6, 0, 7, 4]
[8, 5, 7, 6, 3, 1, 4, 9, 0, 2]
[2, 3, 7, 5, 1, 9, 6, 8, 0, 4]
[5, 7, 8, 6, 1, 4, 2, 9, 3, 0]
[4, 2, 5, 8, 3, 7, 9, 0, 6, 1]


### shuffleとbatch

In [10]:
# batchを使って、指定サイズのミニバッチに分割できる。
# バッチサイズで割りきれずに余った部分は、余った数でバッチ化される。（オプションで切り捨てることも可能）
ds = make_ds().batch(4)
for i in range(LOOPS):
    print_ds(ds)

[array([0, 1, 2, 3], dtype=int64), array([4, 5, 6, 7], dtype=int64), array([8, 9], dtype=int64)]
[array([0, 1, 2, 3], dtype=int64), array([4, 5, 6, 7], dtype=int64), array([8, 9], dtype=int64)]
[array([0, 1, 2, 3], dtype=int64), array([4, 5, 6, 7], dtype=int64), array([8, 9], dtype=int64)]
[array([0, 1, 2, 3], dtype=int64), array([4, 5, 6, 7], dtype=int64), array([8, 9], dtype=int64)]
[array([0, 1, 2, 3], dtype=int64), array([4, 5, 6, 7], dtype=int64), array([8, 9], dtype=int64)]
[array([0, 1, 2, 3], dtype=int64), array([4, 5, 6, 7], dtype=int64), array([8, 9], dtype=int64)]
[array([0, 1, 2, 3], dtype=int64), array([4, 5, 6, 7], dtype=int64), array([8, 9], dtype=int64)]
[array([0, 1, 2, 3], dtype=int64), array([4, 5, 6, 7], dtype=int64), array([8, 9], dtype=int64)]
[array([0, 1, 2, 3], dtype=int64), array([4, 5, 6, 7], dtype=int64), array([8, 9], dtype=int64)]
[array([0, 1, 2, 3], dtype=int64), array([4, 5, 6, 7], dtype=int64), array([8, 9], dtype=int64)]
[array([0, 1, 2, 3], dtype=int

In [11]:
# batch -> shuffle の順序で適用すると、各バッチ単位の固まりでシャッフルされる。
# バッチ内部の値はシャッフルされない。
ds = make_ds().batch(4).shuffle(3)
for i in range(LOOPS):
    print_ds(ds)

[array([4, 5, 6, 7], dtype=int64), array([8, 9], dtype=int64), array([0, 1, 2, 3], dtype=int64)]
[array([8, 9], dtype=int64), array([0, 1, 2, 3], dtype=int64), array([4, 5, 6, 7], dtype=int64)]
[array([4, 5, 6, 7], dtype=int64), array([8, 9], dtype=int64), array([0, 1, 2, 3], dtype=int64)]
[array([0, 1, 2, 3], dtype=int64), array([4, 5, 6, 7], dtype=int64), array([8, 9], dtype=int64)]
[array([4, 5, 6, 7], dtype=int64), array([8, 9], dtype=int64), array([0, 1, 2, 3], dtype=int64)]
[array([4, 5, 6, 7], dtype=int64), array([0, 1, 2, 3], dtype=int64), array([8, 9], dtype=int64)]
[array([4, 5, 6, 7], dtype=int64), array([8, 9], dtype=int64), array([0, 1, 2, 3], dtype=int64)]
[array([8, 9], dtype=int64), array([0, 1, 2, 3], dtype=int64), array([4, 5, 6, 7], dtype=int64)]
[array([8, 9], dtype=int64), array([4, 5, 6, 7], dtype=int64), array([0, 1, 2, 3], dtype=int64)]
[array([0, 1, 2, 3], dtype=int64), array([4, 5, 6, 7], dtype=int64), array([8, 9], dtype=int64)]
[array([0, 1, 2, 3], dtype=int

In [12]:
# shuffle -> batch だと、シャッフルしたデータに対してバッチ化するため、各バッチに含まれる値もシャッフルされる。
ds = make_ds().shuffle(10).batch(4)
for i in range(LOOPS):
    print_ds(ds)

[array([6, 7, 1, 2], dtype=int64), array([8, 0, 9, 3], dtype=int64), array([4, 5], dtype=int64)]
[array([7, 0, 2, 3], dtype=int64), array([1, 9, 6, 8], dtype=int64), array([4, 5], dtype=int64)]
[array([8, 0, 2, 4], dtype=int64), array([5, 1, 6, 7], dtype=int64), array([3, 9], dtype=int64)]
[array([1, 5, 6, 0], dtype=int64), array([9, 8, 3, 7], dtype=int64), array([4, 2], dtype=int64)]
[array([5, 1, 6, 3], dtype=int64), array([2, 0, 8, 4], dtype=int64), array([7, 9], dtype=int64)]
[array([4, 9, 2, 3], dtype=int64), array([0, 6, 1, 7], dtype=int64), array([8, 5], dtype=int64)]
[array([5, 3, 0, 8], dtype=int64), array([9, 1, 4, 6], dtype=int64), array([7, 2], dtype=int64)]
[array([8, 2, 7, 3], dtype=int64), array([6, 5, 4, 0], dtype=int64), array([9, 1], dtype=int64)]
[array([0, 5, 3, 9], dtype=int64), array([8, 1, 2, 4], dtype=int64), array([6, 7], dtype=int64)]
[array([4, 8, 0, 5], dtype=int64), array([6, 7, 9, 2], dtype=int64), array([1, 3], dtype=int64)]
[array([7, 8, 5, 2], dtype=int

In [13]:
# 参考までに、cacheの挙動は、shuffleと組み合わせたときと同じ動き。
# きちんとシャッフルしたければ、cacheはshuffleより先に適用する。
ds = make_ds().shuffle(10).batch(4).cache()
for i in range(LOOPS):
    print_ds(ds)

[array([2, 4, 6, 0], dtype=int64), array([9, 8, 7, 3], dtype=int64), array([1, 5], dtype=int64)]
[array([2, 4, 6, 0], dtype=int64), array([9, 8, 7, 3], dtype=int64), array([1, 5], dtype=int64)]
[array([2, 4, 6, 0], dtype=int64), array([9, 8, 7, 3], dtype=int64), array([1, 5], dtype=int64)]
[array([2, 4, 6, 0], dtype=int64), array([9, 8, 7, 3], dtype=int64), array([1, 5], dtype=int64)]
[array([2, 4, 6, 0], dtype=int64), array([9, 8, 7, 3], dtype=int64), array([1, 5], dtype=int64)]
[array([2, 4, 6, 0], dtype=int64), array([9, 8, 7, 3], dtype=int64), array([1, 5], dtype=int64)]
[array([2, 4, 6, 0], dtype=int64), array([9, 8, 7, 3], dtype=int64), array([1, 5], dtype=int64)]
[array([2, 4, 6, 0], dtype=int64), array([9, 8, 7, 3], dtype=int64), array([1, 5], dtype=int64)]
[array([2, 4, 6, 0], dtype=int64), array([9, 8, 7, 3], dtype=int64), array([1, 5], dtype=int64)]
[array([2, 4, 6, 0], dtype=int64), array([9, 8, 7, 3], dtype=int64), array([1, 5], dtype=int64)]
[array([2, 4, 6, 0], dtype=int

### shuffleとtake/skip

In [14]:
# take/skipでDatasetを任意の場所で区切って取り出せる。
# takeは、Datasetの先頭から指定した要素数のデータを取り出す。
# skipは、Datasetの先頭から指定した要素数をスキップし、その後のデータを取り出す。
# つまり、skip(7)とすれば、8個目の要素以降全てを取り出せる。
# このtake/skipを使って、Datasetをtrain/test用に簡単に分割できる。
ds = make_ds()
ds_x = ds.take(7)
ds_y = ds.skip(7)
for i in range(LOOPS):
    print_ds(ds_x, ds_y)

[0, 1, 2, 3, 4, 5, 6][7, 8, 9]
[0, 1, 2, 3, 4, 5, 6][7, 8, 9]
[0, 1, 2, 3, 4, 5, 6][7, 8, 9]
[0, 1, 2, 3, 4, 5, 6][7, 8, 9]
[0, 1, 2, 3, 4, 5, 6][7, 8, 9]
[0, 1, 2, 3, 4, 5, 6][7, 8, 9]
[0, 1, 2, 3, 4, 5, 6][7, 8, 9]
[0, 1, 2, 3, 4, 5, 6][7, 8, 9]
[0, 1, 2, 3, 4, 5, 6][7, 8, 9]
[0, 1, 2, 3, 4, 5, 6][7, 8, 9]
[0, 1, 2, 3, 4, 5, 6][7, 8, 9]
[0, 1, 2, 3, 4, 5, 6][7, 8, 9]


In [15]:
# shuffleしたデータに対して、take/skipでデータ分割した場合、値を取り出す度に元データ自体がシャッフルされる。
# take/skipで区分けした中でのシャッフル「ではない」ので注意。
# 特に、take/skipでtrain/test用にデータを分割した場合、train/test用データが各エポックごとに混ざってしまうので注意が必要。
ds = make_ds().shuffle(10)
ds_x = ds.take(7)
ds_y = ds.skip(7)
for i in range(LOOPS):
    print_ds(ds_x, ds_y)

[0, 4, 6, 7, 3, 2, 8][4, 7, 9]
[1, 2, 8, 7, 3, 6, 5][8, 7, 9]
[0, 7, 2, 1, 4, 6, 9][8, 1, 7]
[5, 4, 9, 7, 2, 6, 0][9, 3, 7]
[5, 0, 3, 1, 8, 7, 9][0, 7, 4]
[6, 3, 0, 2, 1, 5, 4][4, 9, 5]
[6, 5, 7, 9, 2, 8, 0][9, 4, 7]
[0, 5, 8, 7, 1, 4, 9][1, 6, 0]
[9, 1, 4, 5, 3, 6, 7][1, 2, 5]
[4, 6, 0, 7, 1, 3, 8][4, 8, 3]
[4, 0, 8, 9, 5, 3, 1][4, 8, 9]
[9, 8, 4, 3, 5, 1, 6][2, 4, 5]


In [16]:
# shuffleしたデータを、take/skipで分割後に混ぜたくない場合、shuffleでreshuffle_each_iteration=Falseを指定すれば良い。
# こうしておけば、2回目以降のデータ取り出しでも同じ並び順となるため、take/skipのデータが混ざることは無い。
# ただし、take/skipで分割したそれぞれのデータブロック内でも、毎回同じ並び順になってしまう。
ds = make_ds().shuffle(10, reshuffle_each_iteration=False)
ds_x = ds.take(7)
ds_y = ds.skip(7)
for i in range(LOOPS):
    print_ds(ds_x, ds_y)

[0, 8, 3, 9, 5, 2, 7][4, 1, 6]
[0, 8, 3, 9, 5, 2, 7][4, 1, 6]
[0, 8, 3, 9, 5, 2, 7][4, 1, 6]
[0, 8, 3, 9, 5, 2, 7][4, 1, 6]
[0, 8, 3, 9, 5, 2, 7][4, 1, 6]
[0, 8, 3, 9, 5, 2, 7][4, 1, 6]
[0, 8, 3, 9, 5, 2, 7][4, 1, 6]
[0, 8, 3, 9, 5, 2, 7][4, 1, 6]
[0, 8, 3, 9, 5, 2, 7][4, 1, 6]
[0, 8, 3, 9, 5, 2, 7][4, 1, 6]
[0, 8, 3, 9, 5, 2, 7][4, 1, 6]
[0, 8, 3, 9, 5, 2, 7][4, 1, 6]


In [17]:
# shuffleしたデータを、take/skipで分割後、takeで取得したデータだけを毎回シャッフルしたい場合、
# takeで取得したデータに対して、再度shuffleを適用すればOK。
ds = make_ds().shuffle(10, reshuffle_each_iteration=False)
ds_x = ds.take(7).shuffle(7)  # takeで取得したデータ内でシャッフル
ds_y = ds.skip(7)
for i in range(LOOPS):
    print_ds(ds_x, ds_y)

[5, 2, 7, 3, 0, 8, 4][1, 6, 9]
[4, 8, 7, 5, 3, 0, 2][1, 6, 9]
[8, 3, 5, 4, 0, 2, 7][1, 6, 9]
[0, 3, 2, 7, 4, 8, 5][1, 6, 9]
[4, 3, 2, 8, 5, 7, 0][1, 6, 9]
[3, 0, 8, 2, 4, 7, 5][1, 6, 9]
[3, 0, 4, 7, 5, 2, 8][1, 6, 9]
[4, 5, 7, 8, 2, 0, 3][1, 6, 9]
[3, 7, 5, 8, 0, 2, 4][1, 6, 9]
[7, 0, 5, 8, 3, 2, 4][1, 6, 9]
[8, 5, 2, 3, 0, 4, 7][1, 6, 9]
[4, 0, 2, 8, 3, 7, 5][1, 6, 9]
